# Prohledávání hyperparametrů pro model BERT TINY nad datasetem TREC (fine) 

Tento notebook slouží k nalezení optimálních hyperparametrů nad datasetem TREC (fine) pro model BERRT TINY. Hyperparametry jsou hledány pro původní i augmentovaný dataset pro normální trénink i destilaci.

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou z variant.

## Import knihoven a základní nastavení

In [1]:
from transformers import Trainer, BertTokenizer, BertForSequenceClassification
from datasets import load_from_disk
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Resetování náhodného seedu pro replikovatelnost výsledků.

In [ ]:
base.reset_seed()

Ověření dostupnosti GPU.

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení datasetu a jeho základní předzpracování (tokenizace pomocí učitele).

In [3]:
DATASET = "trec"

In [4]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_fine")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_fine")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_fine")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_fine")
tokenizer = BertTokenizer.from_pretrained("ndavid/autotrain-trec-fine-bert-739422530")

In [5]:
train = train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = all_train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku je pět epochy, maximální 15 epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [6]:
num_epochs = 15
batch_size = 128

In [7]:
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem nad původním datasetem
Definice hledaných hyperparametrů a jejich rozmezí.

In [8]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [9]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Získání předtrénovaného modelu. 

In [10]:
def get_Bert():
    return BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=50)

In [11]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [12]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

Konfigurace trenéra pro jednotlivé tréninky. 

In [13]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [14]:
best_trial_normal = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base",
    n_trials=150
)

[I 2025-03-26 08:36:11,640] A new study created in memory with name: Test-base


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 3}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.805000,3.692317,0.183318,0.011379,0.021918,0.009180
2,3.639400,3.555955,0.180568,0.023551,0.021096,0.008109
3,3.522100,3.433531,0.188818,0.043581,0.023490,0.012242
4,3.404000,3.322597,0.302475,0.074402,0.057608,0.051521
5,3.320300,3.225941,0.372136,0.069201,0.078112,0.064180
6,3.222200,3.145883,0.396884,0.079409,0.085359,0.067539
7,3.141600,3.074114,0.409716,0.095976,0.089143,0.069005
8,3.083900,3.012590,0.421632,0.093745,0.094901,0.075415
9,3.024200,2.960886,0.428048,0.089753,0.099020,0.079480
10,2.980600,2.919415,0.442713,0.088418,0.105445,0.084439


[I 2025-03-26 08:36:59,929] Trial 0 pruned. 


...

[I 2025-03-26 11:00:00,073] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00032676157417641373, 'weight_decay': 0.009000000000000001, 'warmup_steps': 3}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.497700,3.079931,0.354720,0.067490,0.073719,0.058914
2,2.810400,2.494599,0.466544,0.141277,0.123796,0.098519
3,2.286700,2.051042,0.582035,0.274789,0.218098,0.209279
4,1.873700,1.769104,0.670944,0.314039,0.293602,0.279259
5,1.581000,1.561745,0.716774,0.371025,0.353825,0.335868


[I 2025-03-26 11:00:25,314] Trial 149 pruned. 


In [15]:
print(best_trial_normal)

BestRun(run_id='132', objective=0.49578714001038604, hyperparameters={'learning_rate': 0.0004675471848767979, 'weight_decay': 0.01, 'warmup_steps': 4}, run_summary=None)


In [16]:
base.reset_seed()

## Prohledávání s destilací nad původním datasetem
Konfigurace jednotlivých tréninků.

In [17]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.


In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [20]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [21]:
best_trial_distill = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distilation",
    n_trials=150
)

[I 2025-03-26 11:00:25,776] A new study created in memory with name: Distilation


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.417800,2.340572,0.182401,0.013581,0.021644,0.008897
2,2.312900,2.253278,0.179652,0.023548,0.020822,0.007605
3,2.238600,2.177167,0.189734,0.063584,0.023705,0.012673
4,2.169100,2.110003,0.328139,0.072231,0.064126,0.057706
5,2.113300,2.051637,0.387718,0.079887,0.081690,0.065101
6,2.054500,2.001357,0.400550,0.075740,0.085787,0.064986
7,2.006300,1.956808,0.410632,0.093163,0.089287,0.067112
8,1.967700,1.917731,0.429881,0.090969,0.098607,0.077087
9,1.931100,1.884372,0.439047,0.087489,0.103874,0.081015
10,1.903500,1.857554,0.448213,0.105830,0.108713,0.085917


[I 2025-03-26 11:01:18,897] Trial 0 pruned. 


...

[I 2025-03-26 13:25:08,004] Trial 148 finished with value: 0.47070410106521526 and parameters: {'learning_rate': 0.00048236333478817866, 'weight_decay': 0.002, 'warmup_steps': 4, 'lambda_param': 0.0, 'temperature': 2.5}. Best is trial 86 with value: 0.4778879458794155.


Trial 149 with params: {'learning_rate': 0.0003091947441664066, 'weight_decay': 0.004, 'warmup_steps': 3, 'lambda_param': 0.0, 'temperature': 3.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.240600,1.995046,0.376719,0.066179,0.078447,0.063737
2,1.835800,1.630691,0.484876,0.133129,0.135314,0.109298
3,1.508600,1.354026,0.571952,0.224180,0.191599,0.174036
4,1.251700,1.171419,0.660862,0.263684,0.265932,0.249950
5,1.069800,1.043775,0.695692,0.268822,0.291886,0.267089
6,0.920400,0.949116,0.708524,0.299800,0.306229,0.286202
7,0.808500,0.891648,0.708524,0.298573,0.307103,0.284081
8,0.736200,0.853041,0.725940,0.327903,0.335378,0.311706
9,0.675700,0.828287,0.724106,0.330660,0.338123,0.319351
10,0.618100,0.809933,0.742438,0.373351,0.364691,0.349815


[I 2025-03-26 13:25:59,883] Trial 149 pruned. 


In [22]:
print(best_trial_distill)

BestRun(run_id='86', objective=0.4778879458794155, hyperparameters={'learning_rate': 0.00048481023093695626, 'weight_decay': 0.003, 'warmup_steps': 4, 'lambda_param': 0.4, 'temperature': 2.5}, run_summary=None)


Přepočet kroků s ohledem na změnu velikosti datasetu. Ke zmenšení počtu epoch nedochází, augmentovaný dataset je stále relativně malý. 

In [23]:
data_length = len(train_aug)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [24]:
base.reset_seed()

## Prohledávání s normálním tréninkem nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [25]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base-aug_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base-aug_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [26]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [27]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [28]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert()
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [29]:
best_trial_normal_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base-aug",
    n_trials=150
)

[I 2025-03-26 13:26:00,559] A new study created in memory with name: Test-base-aug


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 39}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.932700,2.235381,0.577452,0.251613,0.206702,0.192989
2,1.724400,1.548038,0.703025,0.320607,0.331331,0.310229
3,1.151000,1.272127,0.746104,0.382888,0.401835,0.379462
4,0.842700,1.145172,0.760770,0.441398,0.439341,0.423260
5,0.655500,1.084483,0.766269,0.471065,0.463492,0.449217


[I 2025-03-26 13:27:30,154] Trial 0 pruned. 


...

Trial 136 with params: {'learning_rate': 0.00017198531425921562, 'weight_decay': 0.0, 'warmup_steps': 25}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.775700,1.131487,0.763520,0.413033,0.442284,0.417473
2,0.474400,0.986329,0.778185,0.595577,0.541703,0.545274
3,0.199500,1.004754,0.787351,0.690234,0.623200,0.637726
4,0.101800,1.068647,0.791934,0.721785,0.632165,0.659724


[W 2025-03-26 21:17:28,414] Trial 136 failed with parameters: {'learning_rate': 0.00017198531425921562, 'weight_decay': 0.0, 'warmup_steps': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 250, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2553, in _inner_training_loop
    and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))
KeyboardInterrupt
[W 2025-03-26 21:17:28,416] Trial 136 failed with value None.


KeyboardInterrupt: 

In [ ]:
print(best_trial_normal_aug)

In [30]:
base.reset_seed()

## Prohledávání s destilací nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [31]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill-aug_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill-aug_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [32]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [33]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [34]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert()
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nastavení prohledávání.

In [35]:
best_trial_distill_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-Distill-aug",
    n_trials=150
)

[I 2025-03-26 21:17:39,670] A new study created in memory with name: Test-Distill-aug


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 39, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.867200,1.424818,0.568286,0.233504,0.194461,0.180431
2,1.113900,1.006380,0.703941,0.327727,0.310110,0.289055
3,0.770400,0.844506,0.725940,0.329570,0.344225,0.319838
4,0.591900,0.769759,0.747021,0.409864,0.395760,0.382859
5,0.483900,0.725176,0.754354,0.467335,0.429483,0.422567


[I 2025-03-26 21:19:14,028] Trial 0 pruned. 


...

[I 2025-03-27 05:56:00,613] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00012239133611176964, 'weight_decay': 0.005, 'warmup_steps': 7, 'lambda_param': 1.0, 'temperature': 7.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.308800,0.859274,0.726856,0.342461,0.343372,0.319868
2,0.505900,0.697547,0.767186,0.488707,0.457179,0.454539
3,0.297800,0.650554,0.791934,0.580677,0.538162,0.540940
4,0.203500,0.646560,0.788268,0.648662,0.562095,0.582950
5,0.156800,0.648908,0.797434,0.684762,0.628207,0.641011
6,0.131700,0.639798,0.796517,0.752715,0.643859,0.675339
7,0.115800,0.636609,0.799267,0.762955,0.676112,0.701476
8,0.106300,0.638701,0.800183,0.770711,0.677244,0.705169
9,0.099400,0.644222,0.794684,0.759820,0.669010,0.694522
10,0.093800,0.654706,0.793767,0.752660,0.673049,0.696033


[I 2025-03-27 05:58:58,291] Trial 149 pruned. 


In [36]:
print(best_trial_distill_aug)

BestRun(run_id='92', objective=0.7644517643387146, hyperparameters={'learning_rate': 0.0004922578519032032, 'weight_decay': 0.008, 'warmup_steps': 6, 'lambda_param': 1.0, 'temperature': 4.0}, run_summary=None)
